In [1]:
#import libraries
import torch
import math

from torch import optim
from torch import Tensor

from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

In [2]:
#import data
import dlc_bci as bci
train_input, train_target = bci.load(root =  './data_bci', train=True, one_khz=True)
train_input = train_input 

print(str(type(train_input)), train_input.size())
print(str(type(train_target)), train_target.size())
test_input, test_target = bci.load(root =  './data_bci', train=False, one_khz=True)
print(str(type(test_input)), test_input.size())
print(str(type(test_target)), test_target.size())

F.normalize(train_input, p=2, dim=2) #not sure this normalization does what I want
F.normalize(test_input, p=2, dim=2)

nb_train_signals = train_input.shape[0]
nb_electrodes = train_input.shape[1]
nb_samples_per_signal = train_input.shape[2] #There was a small error in Thibault's code here, corrected here
print('nb_train_signals = ', nb_train_signals, ', nb_electrodes = ', nb_electrodes, ', nb_samples_per_signal = ', nb_samples_per_signal)
mini_batch_size = 32
train_input, test_input, train_target, test_target = Variable(train_input), Variable(test_input), Variable(train_target), Variable(test_target)

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])
nb_train_signals =  316 , nb_electrodes =  28 , nb_samples_per_signal =  500


In [10]:
class Thibaults_model(nn.Module):
    def __init__(self):
        super(Thibaults_model, self).__init__()
        self.conv1 = nn.Conv1d(in_channels = nb_electrodes, out_channels = 32, kernel_size = 3) #Length= nb_samples_per_signal-2 = 498
        self.conv2 = nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3) #Length= nb_samples_per_signal-4 = 496
        self.pool = nn.MaxPool1d(kernel_size = 2) #Length= (nb_samples_per_signal - 4)/2 = 248
        self.conv3 = nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 3) #(Length= nb_samples_per_signal-4)/2 -2 = 246
        self.conv4 = nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 3) #(Length= nb_samples_per_signal-4)/2 -4 = 244
        self.conv5 = nn.Conv1d(in_channels = 64, out_channels = 128, kernel_size = 3) #((Length= nb_samples_per_signal-4)/2 -4)/2 -2 = 120
        self.conv6 = nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size = 3) #((Length= nb_samples_per_signal-4)/2 -4)/2 -4 = 198
        self.conv7 = nn.Conv1d(in_channels = 128, out_channels = 256, kernel_size = 3) #(((Length= nb_samples_per_signal-4)/2 -4)/2 - 4)/2 -2 = 97
        self.conv8 = nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size = 3) #(((Length= nb_samples_per_signal-4)/2 -4)/2 - 4)/2  -4= 95
        self.fc1 = nn.Linear(in_features=14080, out_features=100) #Really not sure, hope so? apparently he recieves 256*32 channels x 55 long signals
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(in_features=100, out_features=100)
        self.fc3 = nn.Linear(in_features=100, out_features=2)
    def forward(self, x):
        x = F.relu(self.conv1(x)) 
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(self.pool(x)))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(self.pool(x)))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(self.pool(x)))
        x = F.relu(self.conv8(x))
        x = x.view(mini_batch_size, -1)
        x = F.relu(self.fc3(F.relu(self.fc2(F.relu(self.fc1(x))))))
        return x

In [11]:
def train_model(model, train_input, train_target):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters()) #No idea what those parameters do
    nb_epochs = 100

    for e in range(0, nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()

######################################################################

def compute_nb_errors(model, data_input, data_target):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output.data, 1)
        for k in range(0, mini_batch_size):
            if data_target.data[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [12]:
model = Thibaults_model()
train_model(model, train_input, train_target)
nb_test_errors = compute_nb_errors(model, test_input, test_target)
print('test error Net {:0.2f}%% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                      nb_test_errors, test_input.size(0)))


RuntimeError: invalid argument 4: out of range at /opt/conda/conda-bld/pytorch_1518243271935/work/torch/lib/TH/generic/THTensor.c:440